<a href="https://colab.research.google.com/github/saffarizadeh/INSY5378/blob/main/Backpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://kambizsaffari.com/Logo/College_of_Business.cmyk-hz-lg.png" width="500px"/>

# *INSY 5378 - Advanced AI*

# **Backpropagation with Matrix Calculus**

Instructor: Dr. Kambiz Saffari

---


## What Are We Trying to Do?

A neural network is just a machine that takes in numbers, does some math, and spits out a prediction. The "math" involves multiplying your input by a bunch of numbers called **weights**. At first, these weights are random, so the predictions are garbage.

**Training** is the process of adjusting these weights so the predictions get better. But how do we know which way to adjust each weight? That's where **backpropagation** comes in.

Here's the intuition:

1. **We make a prediction** — Feed some data through the network.
2. **We measure how wrong we are** — Compare the prediction to the correct answer. The difference is called the **loss**.
3. **We figure out who's to blame** — For each weight in the network, we ask: "If I nudged this weight up a tiny bit, would the loss go up or down? By how much?" This is the **gradient**.
4. **We adjust the weights** — Nudge each weight in the direction that makes the loss smaller.
5. **Repeat** — Do this thousands of times until the loss is small and the predictions are good.

The tricky part is step 3. A neural network might have millions of weights. We need an efficient way to compute the gradient for *all* of them at once. That's what backpropagation does — it uses the **chain rule** from calculus to propagate the error backward through the network, layer by layer, computing all the gradients in one pass.

The math below looks intimidating, but it's just a precise way of answering: **"How much does each weight contribute to the error, and in which direction?"**

## The Model We're Using

We're using the simplest possible neural network that's still interesting: **two layers**.

```
Input (n features)
    ↓
[Layer 1] — multiply by weights, add bias, apply ReLU
    ↓
Hidden (h neurons)
    ↓
[Layer 2] — multiply by weights, add bias
    ↓
Output (m values)
```

**What's ReLU?** It's a simple function: if the number is negative, make it zero. Otherwise, keep it. That's it. Written as: $\text{relu}(x) = \max(0, x)$.

**Why ReLU?** Without it, stacking layers would be pointless — two linear transformations in a row is just one linear transformation. ReLU adds non-linearity, which lets the network learn complex patterns.

In Keras, this model looks like:

```python
import keras
from keras import layers

model = keras.Sequential([
    layers.Dense(h, activation='relu', input_shape=(n,)),  # Layer 1: n → h
    layers.Dense(m)                                         # Layer 2: h → m
])

model.compile(optimizer='sgd', loss='mse')
model.fit(X, Y, epochs=1000, batch_size=32)
```

That's it. Keras hides all the backpropagation math from you. But under the hood, it's doing exactly what we derive below.

## 1. Definitions

| Symbol | Shape | Description |
|--------|-------|-------------|
| $X$ | $B \times n$ | Input batch ($B$ samples, $n$ features) |
| $Y$ | $B \times m$ | Targets |
| $W_1$ | $n \times h$ | First layer weights |
| $b_1$ | $1 \times h$ | First layer bias |
| $W_2$ | $h \times m$ | Second layer weights |
| $b_2$ | $1 \times m$ | Second layer bias |

## 2. Forward Pass

$$Z_1 = XW_1 + b_1 \in \mathbb{R}^{B \times h}$$

$$A_1 = \text{relu}(Z_1) \in \mathbb{R}^{B \times h}$$

where $\text{relu}(Z_1)_{ij} = \max(0, (Z_1)_{ij})$

$$\hat{Y} = A_1 W_2 + b_2 \in \mathbb{R}^{B \times m}$$

## 3. Loss Function

Mean squared error:

$$\ell = \frac{1}{2B}\|\hat{Y} - Y\|_F^2 \in \mathbb{R}$$

where $\|\cdot\|_F$ is the Frobenius norm: $\|M\|_F^2 = \sum_{ij} M_{ij}^2$

**Why $\frac{1}{2B}$?**
- The $\frac{1}{B}$ averages over the batch, so the gradient magnitude doesn't depend on batch size.
- The $\frac{1}{2}$ cancels the 2 from differentiating $x^2$, keeping the math cleaner.

## 4. Matrix Calculus Identity

Let $A \in \mathbb{R}^{p \times q}$, $B \in \mathbb{R}^{q \times r}$, $Z = AB \in \mathbb{R}^{p \times r}$, and $\ell = f(Z) \in \mathbb{R}$ where $f$ is any differentiable scalar-valued function.

Given $\frac{\partial \ell}{\partial Z} \in \mathbb{R}^{p \times r}$:

$$\frac{\partial \ell}{\partial A} = \frac{\partial \ell}{\partial Z} B^\top \in \mathbb{R}^{p \times q}$$

$$\frac{\partial \ell}{\partial B} = A^\top \frac{\partial \ell}{\partial Z} \in \mathbb{R}^{q \times r}$$

The transpose always goes on the factor you're **not** differentiating with respect to.

## 5. Backward Pass

### 5.1 Gradient of loss: $\frac{\partial \ell}{\partial W_2}$

Chain:
$$\frac{\partial \ell}{\partial W_2} = \frac{\partial \ell}{\partial \hat{Y}} \cdot \frac{\partial \hat{Y}}{\partial W_2}$$

Using the identity $\frac{\partial}{\partial M}\frac{1}{2}\|M\|_F^2 = M$:

$$\frac{\partial \ell}{\partial \hat{Y}} = \frac{1}{B}(\hat{Y} - Y) \in \mathbb{R}^{B \times m}$$

Since $\hat{Y} = A_1 W_2 + b_2$, applying the matrix calculus identity:

$$\boxed{\frac{\partial \ell}{\partial W_2} = A_1^\top \frac{\partial \ell}{\partial \hat{Y}} = A_1^\top \cdot \frac{1}{B}(\hat{Y} - Y) \in \mathbb{R}^{h \times m}}$$

### 5.2 Gradient of loss: $\frac{\partial \ell}{\partial b_2}$

Chain:
$$\frac{\partial \ell}{\partial b_2} = \frac{\partial \ell}{\partial \hat{Y}} \cdot \frac{\partial \hat{Y}}{\partial b_2}$$

Since $b_2$ is broadcast across all $B$ samples, we sum over the batch:

$$\boxed{\frac{\partial \ell}{\partial b_2} = \mathbf{1}^\top \frac{\partial \ell}{\partial \hat{Y}} = \mathbf{1}^\top \cdot \frac{1}{B}(\hat{Y} - Y) \in \mathbb{R}^{1 \times m}}$$

where $\mathbf{1} \in \mathbb{R}^{B \times 1}$ is a column vector of ones.

### 5.3 Gradient of loss: $\frac{\partial \ell}{\partial W_1}$

Chain:
$$\frac{\partial \ell}{\partial W_1} = \frac{\partial \ell}{\partial \hat{Y}} \cdot \frac{\partial \hat{Y}}{\partial A_1} \cdot \frac{\partial A_1}{\partial Z_1} \cdot \frac{\partial Z_1}{\partial W_1}$$

Step by step:

**Through the second layer ($\hat{Y} = A_1 W_2 + b_2$):**
$$\frac{\partial \ell}{\partial A_1} = \frac{\partial \ell}{\partial \hat{Y}} W_2^\top = \frac{1}{B}(\hat{Y} - Y) W_2^\top \in \mathbb{R}^{B \times h}$$

**Through relu ($A_1 = \text{relu}(Z_1)$):**
$$\frac{\partial \ell}{\partial Z_1} = \frac{\partial \ell}{\partial A_1} \odot \mathbb{1}[Z_1 > 0] \in \mathbb{R}^{B \times h}$$

where $\odot$ is element-wise multiplication (Hadamard product) and $\mathbb{1}[Z_1 > 0]$ is 1 where $Z_1 > 0$, else 0.

**Through the first layer ($Z_1 = XW_1 + b_1$):**
$$\boxed{\frac{\partial \ell}{\partial W_1} = X^\top \frac{\partial \ell}{\partial Z_1} = X^\top \left[\left(\frac{1}{B}(\hat{Y} - Y) W_2^\top\right) \odot \mathbb{1}[Z_1 > 0]\right] \in \mathbb{R}^{n \times h}}$$

### 5.4 Gradient of loss: $\frac{\partial \ell}{\partial b_1}$

Chain:
$$\frac{\partial \ell}{\partial b_1} = \frac{\partial \ell}{\partial \hat{Y}} \cdot \frac{\partial \hat{Y}}{\partial A_1} \cdot \frac{\partial A_1}{\partial Z_1} \cdot \frac{\partial Z_1}{\partial b_1}$$

We already computed $\frac{\partial \ell}{\partial Z_1}$ above. Since $b_1$ is broadcast across all $B$ samples:

$$\boxed{\frac{\partial \ell}{\partial b_1} = \mathbf{1}^\top \frac{\partial \ell}{\partial Z_1} = \mathbf{1}^\top \left[\left(\frac{1}{B}(\hat{Y} - Y) W_2^\top\right) \odot \mathbb{1}[Z_1 > 0]\right] \in \mathbb{R}^{1 \times h}}$$

## 6. Summary of Gradients

Let $\delta_2 = \frac{1}{B}(\hat{Y} - Y)$ and $\delta_1 = (\delta_2 W_2^\top) \odot \mathbb{1}[Z_1 > 0]$. Then:

| Gradient | Formula | Shape |
|----------|---------|-------|
| $\frac{\partial \ell}{\partial W_2}$ | $A_1^\top \delta_2$ | $h \times m$ |
| $\frac{\partial \ell}{\partial b_2}$ | $\mathbf{1}^\top \delta_2$ | $1 \times m$ |
| $\frac{\partial \ell}{\partial W_1}$ | $X^\top \delta_1$ | $n \times h$ |
| $\frac{\partial \ell}{\partial b_1}$ | $\mathbf{1}^\top \delta_1$ | $1 \times h$ |

## 7. Update Rule

$$W_2 \leftarrow W_2 - \eta \frac{\partial \ell}{\partial W_2}$$

$$b_2 \leftarrow b_2 - \eta \frac{\partial \ell}{\partial b_2}$$

$$W_1 \leftarrow W_1 - \eta \frac{\partial \ell}{\partial W_1}$$

$$b_1 \leftarrow b_1 - \eta \frac{\partial \ell}{\partial b_1}$$

where $\eta$ is the learning rate.

## 8. Implementation

In [1]:
import numpy as np

def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return (Z > 0).astype(float)

def forward(X, W1, b1, W2, b2):
    Z1 = X @ W1 + b1
    A1 = relu(Z1)
    Y_hat = A1 @ W2 + b2
    return Z1, A1, Y_hat

def loss(Y_hat, Y):
    B = Y.shape[0]
    return (1 / (2 * B)) * np.sum((Y_hat - Y) ** 2)

def backward(X, Y, Z1, A1, Y_hat, W2):
    B = X.shape[0]

    # delta_2 = dL/dY_hat
    delta_2 = (1 / B) * (Y_hat - Y)

    # Second layer gradients
    dW2 = A1.T @ delta_2
    db2 = np.sum(delta_2, axis=0, keepdims=True)

    # delta_1 = dL/dZ1
    delta_1 = (delta_2 @ W2.T) * relu_derivative(Z1)

    # First layer gradients
    dW1 = X.T @ delta_1
    db1 = np.sum(delta_1, axis=0, keepdims=True)

    return dW1, db1, dW2, db2

def update(W1, b1, W2, b2, dW1, db1, dW2, db2, lr):
    W1 = W1 - lr * dW1
    b1 = b1 - lr * db1
    W2 = W2 - lr * dW2
    b2 = b2 - lr * db2
    return W1, b1, W2, b2

## 9. Example

In [2]:
# Dimensions
B = 32   # batch size
n = 10   # input features
h = 20   # hidden units
m = 5    # output units

# Initialize
np.random.seed(42)
X = np.random.randn(B, n)
Y = np.random.randn(B, m)
W1 = np.random.randn(n, h) * 0.01
b1 = np.zeros((1, h))
W2 = np.random.randn(h, m) * 0.01
b2 = np.zeros((1, m))

# Training loop
lr = 0.1
for i in range(1000):
    Z1, A1, Y_hat = forward(X, W1, b1, W2, b2)
    L = loss(Y_hat, Y)
    dW1, db1, dW2, db2 = backward(X, Y, Z1, A1, Y_hat, W2)
    W1, b1, W2, b2 = update(W1, b1, W2, b2, dW1, db1, dW2, db2, lr)
    if i % 100 == 0:
        print(f"Iteration {i}, Loss: {L:.6f}")

Iteration 0, Loss: 2.514020
Iteration 100, Loss: 1.454426
Iteration 200, Loss: 0.628180
Iteration 300, Loss: 0.286071
Iteration 400, Loss: 0.104141
Iteration 500, Loss: 0.044479
Iteration 600, Loss: 0.022456
Iteration 700, Loss: 0.012840
Iteration 800, Loss: 0.008105
Iteration 900, Loss: 0.005215
